# The Battle of Neighborhoods

## Importing libraries 

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [17]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 3.3MB/s ta 0:00:011


In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import geocoder
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as color

## Scrap data from Wikipedia page into a DataFrame¶

In [19]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
mumbai_df = pd.DataFrame({"Neighborhood": neighborhoodList})
mumbai_df = mumbai_df.iloc[1:]
mumbai_df.head()

,Neighborhood
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"
5,Amrut Nagar


In [20]:
# print the number of rows of the dataframe
mumbai_df.shape

(134, 1)

##  Get the geographical coordinates for each neighborhood

In [21]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [22]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in mumbai_df["Neighborhood"].tolist() ]

In [23]:
coordinatesrdinates

[[19.166149990244833, 72.85940996038701],
 [18.97628000000003, 72.82615000000004],
 [18.964337724315293, 72.80783777682117],
 [18.940170000000023, 72.83483000000007],
 [19.145160000000033, 72.84674000000007],
 [19.026140000000055, 72.86645000000004],
 [19.042830000000038, 72.92734000000007],
 [19.10301000000004, 72.89935000000008],
 [18.920250012967855, 72.82613003292406],
 [19.062940000000026, 72.92663000000005],
 [18.935490000000073, 72.83941000000004],
 [19.054370000000063, 72.84017000000006],
 [19.062080000000037, 72.86844000000008],
 [19.167850000000044, 72.83292000000006],
 [18.95130000000006, 72.82930000000005],
 [19.204780000000028, 72.84128000000004],
 [18.969030000000032, 72.80384000000004],
 [18.980740000000026, 72.84072000000003],
 [19.138910000000067, 72.93817000000007],
 [18.949704950394594, 72.8273265646136],
 [18.996790000000033, 72.84654000000006],
 [19.11017000000004, 72.89171000000005],
 [18.985870000000034, 72.83108000000004],
 [18.947690000000023, 72.82697000000007

In [29]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
mumbai_df['Latitude'] = df_coords['Latitude']
mumbai_df['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
mumbai_df = mumbai_df.dropna()
mumbai_df = mumbai_df.reset_index(drop=True)
print(mumbai_df.shape)
mumbai_df

(133, 3)


,Neighborhood,Latitude,Longitude
0,Aarey Milk Colony,18.976280,72.826150
1,Agripada,18.964338,72.807838
2,Altamount Road,18.940170,72.834830
3,"Amboli, Mumbai",19.145160,72.846740
4,Amrut Nagar,19.026140,72.866450
5,Antop Hill,19.042830,72.927340
6,Anushakti Nagar,19.103010,72.899350
7,Asalfa,18.920250,72.826130
8,Badhwar Park,19.062940,72.926630
9,Baiganwadi,18.935490,72.839410


In [30]:
# save the DataFrame as CSV file
mumbai_df.to_csv("mumbai_df.csv", index=False)

## Create a map of Mumbai, India with neighborhoods superimposed on top

In [31]:
# get the coordinates of mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 19.0759899, 72.8773928.


In [34]:
# create map of mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [35]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

## Use the Foursquare API to explore the neighborhoods

### Exploring venues within 3000 meters

In [39]:
# define Foursquare Credentials and Version
CLIENT_ID = '4NUCBBIRCJ4HBDBEYJCYQFVMYHW3YHBFSWSUQRHB2B24G2CS' # your Foursquare ID
CLIENT_SECRET = 'NYAU2CIRBL0XW03OF3SDBK0JO3DVAJHDECC413MQM3FRBJI4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 3000
venues = []

for lat, long, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [40]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(11752, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aarey Milk Colony,18.97628,72.82615,Mahalaxmi Race Course (Royal Western India Tur...,18.980535,72.818588,Club House
1,Aarey Milk Colony,18.97628,72.82615,Tote On The Turf,18.980266,72.820294,Nightclub
2,Aarey Milk Colony,18.97628,72.82615,Willingdon Sports Club,18.976925,72.815256,Golf Course
3,Aarey Milk Colony,18.97628,72.82615,Haji Ali Sea Face,18.978995,72.811854,Scenic Lookout
4,Aarey Milk Colony,18.97628,72.82615,Richardson and Cruddas,18.966491,72.832793,Music Venue


In [41]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,100,100,100,100,100,100
Agripada,73,73,73,73,73,73
Altamount Road,100,100,100,100,100,100
"Amboli, Mumbai",100,100,100,100,100,100
Amrut Nagar,100,100,100,100,100,100
Antop Hill,23,23,23,23,23,23
Anushakti Nagar,100,100,100,100,100,100
Asalfa,59,59,59,59,59,59
Badhwar Park,36,36,36,36,36,36


In [42]:
#unique categories
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

There are 205 uniques categories.


array(['Club House', 'Nightclub', 'Golf Course', 'Scenic Lookout',
       'Music Venue', 'Indian Restaurant', 'History Museum', 'Hotel',
       'Middle Eastern Restaurant', 'Shopping Mall', 'Bakery', 'Lounge',
       'Juice Bar', 'Restaurant', 'Chinese Restaurant', 'Deli / Bodega',
       'Comedy Club', 'Pub', 'Cosmetics Shop', 'Multiplex',
       'Toy / Game Store', 'Hotel Bar', 'Pizza Place', 'Cupcake Shop',
       'Ice Cream Shop', 'Sandwich Place', 'Donut Shop', 'Café',
       'Brewery', 'Bar', 'Spa', 'Stadium', 'Gym / Fitness Center',
       'Gourmet Shop', 'Dessert Shop', 'Seafood Restaurant',
       'Maharashtrian Restaurant', 'Clothing Store', 'Bookstore',
       'Coffee Shop', 'Planetarium', "Men's Store",
       'Vegetarian / Vegan Restaurant', 'Molecular Gastronomy Restaurant',
       'Beach', 'Asian Restaurant', 'Italian Restaurant', 'Park',
       'Japanese Restaurant', 'Furniture / Home Store'], dtype=object)

In [43]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## Analyze each neighborhood

In [44]:
# one hot encoding
mumbai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

print(mumbai_onehot.shape)
mumbai_onehot.head()

(11752, 206)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Entertainment,...,Theme Park,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Shop,Women's Store,Zoo
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
mumbai_grouped = mumbai_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped

(133, 206)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Entertainment,...,Theme Park,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Shop,Women's Store,Zoo
0,Aarey Milk Colony,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.010000,0.00,0.0,0.000000,0.010000,0.00,0.000000,0.000000,0.00
1,Agripada,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.027397,0.0,...,0.000000,0.000000,0.00,0.0,0.000000,0.013699,0.00,0.000000,0.000000,0.00
2,Altamount Road,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.010000,0.0,...,0.000000,0.000000,0.00,0.0,0.010000,0.000000,0.00,0.000000,0.000000,0.00
3,"Amboli, Mumbai",0.000000,0.0,0.00,0.00,0.010000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.01,0.0,0.000000,0.010000,0.00,0.000000,0.020000,0.00
4,Amrut Nagar,0.000000,0.0,0.00,0.00,0.000000,0.0,0.010000,0.000000,0.0,...,0.000000,0.000000,0.01,0.0,0.020000,0.030000,0.00,0.000000,0.010000,0.00
5,Antop Hill,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,Anushakti Nagar,0.010000,0.0,0.00,0.01,0.020000,0.0,0.000000,0.000000,0.0,...,0.010000,0.010000,0.00,0.0,0.000000,0.020000,0.00,0.000000,0.000000,0.00
7,Asalfa,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.016949,0.0,...,0.000000,0.000000,0.00,0.0,0.016949,0.000000,0.00,0.000000,0.000000,0.00
8,Badhwar Park,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.0,0.000000,0.083333,0.00,0.000000,0.000000,0.00
9,Baiganwadi,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.010000,0.0,...,0.000000,0.000000,0.00,0.0,0.010000,0.000000,0.00,0.000000,0.000000,0.00


In [47]:
len(mumbai_grouped[mumbai_grouped["Shopping Mall"] > 0])

41

## Create a new DataFrame for Shopping Mall data only

In [48]:
mumbai_mall = mumbai_grouped[["Neighborhoods","Shopping Mall"]]

In [49]:
mumbai_mall.head()

,Neighborhoods,Shopping Mall
0,Aarey Milk Colony,0.02
1,Agripada,0.00
2,Altamount Road,0.00
3,"Amboli, Mumbai",0.00
4,Amrut Nagar,0.01


## Clustering neighborhood

### Run k-means to cluster the neighborhoods in Mumbai into 4  clusters.

In [50]:
# set number of clusters
kclusters = 4

mumbai_clustering = mumbai_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 3, 0, 1, 0, 0, 0], dtype=int32)

In [51]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mumbai_merged = mumbai_mall.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

In [52]:
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aarey Milk Colony,0.02,1
1,Agripada,0.00,0
2,Altamount Road,0.00,0
3,"Amboli, Mumbai",0.00,0
4,Amrut Nagar,0.01,3


In [53]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head() # check the last columns!

(133, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.02,1,18.976280,72.826150
1,Agripada,0.00,0,18.964338,72.807838
2,Altamount Road,0.00,0,18.940170,72.834830
3,"Amboli, Mumbai",0.00,0,19.145160,72.846740
4,Amrut Nagar,0.01,3,19.026140,72.866450


In [54]:
# sort the results by Cluster Labels
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

(133, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
66,Kemps Corner,0.000000,0,19.084250,72.826870
87,Manori,0.000000,0,18.946660,72.824560
86,"Mandvi, Mumbai",0.000000,0,19.209380,72.783150
85,Malad,0.000000,0,18.952410,72.841480
83,Mahul,0.000000,0,18.950370,72.795760
81,"Maheshwari Udyan, Mumbai",0.000000,0,19.040730,72.843050
80,"Mahalaxmi, Mumbai",0.000000,0,19.027600,72.855220
79,Madh Island,0.000000,0,18.946860,72.828890
78,Lokhandwala Complex,0.000000,0,19.146810,72.790670
77,Lohar Chawl,0.000000,0,19.142292,72.828632


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [color.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Analyze the cluster

### Cluster 1

In [59]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
66,Kemps Corner,0.0,0,19.084250,72.826870
87,Manori,0.0,0,18.946660,72.824560
86,"Mandvi, Mumbai",0.0,0,19.209380,72.783150
85,Malad,0.0,0,18.952410,72.841480
83,Mahul,0.0,0,18.950370,72.795760
81,"Maheshwari Udyan, Mumbai",0.0,0,19.040730,72.843050
80,"Mahalaxmi, Mumbai",0.0,0,19.027600,72.855220
79,Madh Island,0.0,0,18.946860,72.828890
78,Lokhandwala Complex,0.0,0,19.146810,72.790670
77,Lohar Chawl,0.0,0,19.142292,72.828632


### Cluster 2

In [60]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
6,Anushakti Nagar,0.020000,1,19.10301,72.89935
14,Bhuleshwar,0.020000,1,19.20478,72.84128
109,Royal Opera House (Mumbai),0.023256,1,19.19574,72.83531
103,Pali Village,0.020000,1,18.99570,72.83913
16,Breach Candy,0.020000,1,18.98074,72.84072
113,Sewri,0.020000,1,19.16045,72.83651
118,Sunder Nagar,0.020000,1,18.97254,72.81478
0,Aarey Milk Colony,0.020000,1,18.97628,72.82615
47,Gavangaon,0.020000,1,18.98109,72.84100
30,Cumbala Hill,0.020000,1,18.99515,72.83273


### Cluster 3

In [61]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Byculla,0.037736,2,19.13891,72.93817
59,JB Nagar,0.040000,2,19.19822,72.96171
57,I.C. Colony,0.037975,2,19.01639,72.82901
131,Western Suburbs (Mumbai),0.030000,2,19.19021,72.85365
111,SPDC Colony,0.030000,2,19.01499,72.83845
123,Uran,0.051282,2,26.86634,80.93884
84,Malabar Hill,0.030000,2,19.18655,72.84842
12,Bandra Kurla Complex,0.030000,2,19.16785,72.83292
105,Powai,0.033898,2,19.01420,72.82776


In [ ]:
### 